In [ ]:
import pandas as pd
import numpy as np
import os
from tkinter import Tk, Label, Button, filedialog, Listbox, Text, Scrollbar, END, MULTIPLE
from tkinter.messagebox import showinfo
from collections import Counter

pd.set_option('display.max_columns', 100)

def calc_revenue(df, month=None, store_name=None):
    df.columns = [col.replace('\n', '') for col in df.columns]
    condition = df['狀態'].apply(lambda x: x.split('\n')[0] not in ['取消訂單', '逾期未取件']) 
    df = df.loc[condition]
    
    if month:
        condition = df['訂購日期'].apply(lambda x: x.split('/')[1] in month)
        df = df.loc[condition]

    if store_name:
        condition = df['賣場名稱'].apply(lambda x: x in store_name)
        df = df.loc[condition]

    revenue = df['小計(A)'].apply(lambda x: x.replace(',', '')).astype(int).sum()
    return revenue


def read_data(df_paths):
    df_list = [pd.read_excel(df_path, skiprows=2) for df_path in df_paths]
    return pd.concat(df_list, axis=0)


def select_files():
    file_paths = filedialog.askopenfilenames(title="選擇檔案", filetypes=[("Excel files", "*.xlsx *.xls")])
    if file_paths:
        file_listbox.delete(0, END)
        selected_files.clear()
        selected_files.extend(file_paths)

        for file in file_paths:
            file_listbox.insert(END, os.path.basename(file))


def calculate_revenue():
    if not selected_files:
        showinfo("提示", "未選擇任何檔案")
        return
    
    selected_months = [month_listbox.get(i) for i in month_listbox.curselection()]
    month = [str(month).zfill(2) for month in selected_months]

    store_name = None
    df = read_data(selected_files)
    revenue = calc_revenue(df=df, month=month, store_name=store_name)
    showinfo("營收結果", f"總營收: {revenue}")


def count_buyers(df):
    df_filtered = df.loc[df.iloc[:, 0].map(lambda x: x != '下架日/開售日')]
    df_value = df_filtered.iloc[:, 4:].replace({'\u3000': ' '}, regex=True)
    return Counter(df_value.stack())


def unique_buyers(df):
    df_filtered = df.loc[df.iloc[:, 0].map(lambda x: x != '下架日/開售日')]
    df_value = df_filtered.iloc[:, 4:].replace({'\u3000': ' '}, regex=True)
    unique_names = list(df_value.stack().unique())
    return unique_names, len(unique_names)


def sorted_unique_buyers(df):
    unique_names, count = unique_buyers(df)
    return sorted(unique_names), count


def calculate_buyer_totals(df):
    df_filtered = df.loc[df.iloc[:, 0].map(lambda x: x != '下架日/開售日')]
    prices = df_filtered.iloc[:, 3]
    buyers = df_filtered.iloc[:, 4:]
    buyer_totals = {}

    for i, row in buyers.iterrows():
        price = int(prices[i])
        for buyer in row.dropna():
            if buyer in buyer_totals:
                buyer_totals[buyer] += price
            else:
                buyer_totals[buyer] = price
    return buyer_totals


def sorted_buyer_totals(df):
    buyer_totals = calculate_buyer_totals(df)
    return dict(sorted(buyer_totals.items(), key=lambda item: item[1], reverse=True))


def display_buyer_count():
    if not selected_files:
        showinfo("提示", "未選擇任何檔案")
        return
    
    df = pd.read_excel(selected_files[0], header=None)
    buyer_count = count_buyers(df)
    output_text.delete("1.0", END)
    output_text.insert(END, f"每個買家出現次數:\n{buyer_count}")


def display_unique_buyers():
    if not selected_files:
        showinfo("提示", "未選擇任何檔案")
        return
    
    df = pd.read_excel(selected_files[0], header=None)
    unique_names, total = unique_buyers(df)
    output_text.delete("1.0", END)
    output_text.insert(END, f"不重複的買家名及總數:\n總數: {total}\n{unique_names}")


def display_sorted_unique_buyers():
    if not selected_files:
        showinfo("提示", "未選擇任何檔案")
        return
    
    df = pd.read_excel(selected_files[0], header=None)
    sorted_names, total = sorted_unique_buyers(df)
    output_text.delete("1.0", END)
    output_text.insert(END, f"排序後不重複的買家名及總數:\n總數: {total}\n{sorted_names}\n")


def display_buyer_totals():
    if not selected_files:
        showinfo("提示", "未選擇任何檔案")
        return
    
    df = pd.read_excel(selected_files[0], header=None)
    buyer_totals = calculate_buyer_totals(df)
    output_text.delete("1.0", END)
    output_text.insert(END, f"每個買家的金額:\n{buyer_totals}")


def display_sorted_buyer_totals():
    if not selected_files:
        showinfo("提示", "未選擇任何檔案")
        return
    
    df = pd.read_excel(selected_files[0], header=None)
    sorted_totals = sorted_buyer_totals(df)
    output_text.delete("1.0", END)
    output_text.insert(END, f"排序後每個買家的金額:\n{sorted_totals}")


def main_menu():
    for widget in root.winfo_children():
        widget.destroy()
    
    Label(root, text="請選擇功能").pack(pady=20)
    Button(root, text="計算營收", command=revenue_menu).pack(pady=10)
    Button(root, text="統計買家", command=buyer_stats_menu).pack(pady=10)


def revenue_menu():
    for widget in root.winfo_children():
        widget.destroy()
    
    Label(root, text="請選擇月份:").pack(pady=10)
    
    global month_listbox, file_listbox
    month_listbox = Listbox(root, selectmode='extended', width=20, height=12, exportselection=False)
    months = [str(i) for i in range(1, 13)]
    for month in months:
        month_listbox.insert(END, month)
    month_listbox.pack()

    Button(root, text="選擇檔案", command=select_files).pack(pady=10)
    Label(root, text="已選擇的檔案:").pack()
    file_listbox = Listbox(root, width=50, height=8)
    file_listbox.pack()
    Button(root, text="開始計算", command=calculate_revenue).pack(pady=20)
    Button(root, text="返回主選單", command=main_menu).pack(pady=10)


def buyer_stats_menu():
    for widget in root.winfo_children():
        widget.destroy()
    
    Button(root, text="選擇檔案", command=select_files).pack(pady=10)
    Label(root, text="已選擇的檔案:").pack()
    global file_listbox
    file_listbox = Listbox(root, width=50, height=8)
    file_listbox.pack()

    Button(root, text="統計每個買家的出現次數", command=display_buyer_count).pack(pady=5)
    Button(root, text="顯示不重複的買家名及總數", command=display_unique_buyers).pack(pady=5)
    Button(root, text="顯示排序後不重複的買家名及總數", command=display_sorted_unique_buyers).pack(pady=5)
    Button(root, text="顯示每個買家的金額", command=display_buyer_totals).pack(pady=5)
    Button(root, text="顯示排序後每個買家的金額", command=display_sorted_buyer_totals).pack(pady=5)
    
    global output_text
    output_text = Text(root, width=50, height=15)
    output_text.pack(pady=10)
    Button(root, text="返回主選單", command=main_menu).pack(pady=10)


root = Tk()
root.title("營收與買家統計工具")
selected_files = []

main_menu()

root.update_idletasks()
root.minsize(root.winfo_reqwidth(), root.winfo_reqheight())
root.mainloop()
